In [1]:
%cd /home/nahuel/ecg/generalization/models
%load_ext autoreload
%autoreload 2

/home/nahuel/ecg/generalization/models


/home/nahuel/ecg/generalization/generalization/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
import sys
import os

# Agregar la carpeta que contiene 'data.py' al sys.path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../data')))

In [3]:
from model import select_model, ECGModel
from data import load_data, get_shot

## S2S

https://arxiv.org/pdf/1812.07421v2

Comandos utilizados para lanzar experimentos (desde la terminal): 

Modelo entrenado con MIT

```bash
python models/train.py --experiment_name MIT-s2s --dataset MIT-BIH --model Seq2Seq --path models/saved/MIT-BIH/Seq2Seq --return_sequences
```


Modelo entrenado con INCART

```bash
python models/train.py --experiment_name IncartDB-s2s --dataset INCART --model Seq2Seq --path models/saved/INCART/Seq2Seq --return_sequences
```

In [4]:
model_name = "Seq2Seq"
model_module = select_model(model_name, n_classes=4)

for dataset_name in ["MIT-BIH", "INCART"]:
	print("LOADING MODEL TRAINED WITH " + dataset_name + "...")
	path = "/home/nahuel/ecg/generalization/models/saved/"+dataset_name+"/"+model_name+"/best_model.ckpt"
	model = ECGModel.load_from_checkpoint(path, model=model_module)

	for dataset_name in ["MIT-BIH", "INCART"]:
		
		train_dataloader, val_dataloader, test_dataloader = load_data(dataset_name, return_sequences=True)
		print("Evaluating model with " + dataset_name)
		model.evaluate(test_dataloader, "cpu")

LOADING MODEL TRAINED WITH MIT-BIH...
Error in Tzone or Pzone: attempt to get argmax of an empty sequence
Error in Tzone or Pzone: attempt to get argmax of an empty sequence
Error in Tzone or Pzone: attempt to get argmax of an empty sequence
Error in Tzone or Pzone: attempt to get argmax of an empty sequence
Error in Tzone or Pzone: attempt to get argmax of an empty sequence
Error in Tzone or Pzone: attempt to get argmax of an empty sequence
Error in Tzone or Pzone: attempt to get argmax of an empty sequence
Records processed!
Evaluating model with MIT-BIH
Accuracy own: 0.9972
Accuracy: 0.9972
F1 Score: 0.9884
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     18170
           1       0.92      1.00      0.96       560
           2       1.00      1.00      1.00      1400
           3       1.00      1.00      1.00       110

    accuracy                           1.00     20240
   macro avg       0.98      1.00      0.99     20240
we

/home/nahuel/ecg/generalization/generalization/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/nahuel/ecg/generalization/generalization/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/nahuel/ecg/generalization/generalization/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(av

Error in Tzone or Pzone: attempt to get argmax of an empty sequence
Error in Tzone or Pzone: attempt to get argmax of an empty sequence
Error in Tzone or Pzone: attempt to get argmax of an empty sequence
Error in Tzone or Pzone: attempt to get argmax of an empty sequence
Error in Tzone or Pzone: attempt to get argmax of an empty sequence
Error in Tzone or Pzone: attempt to get argmax of an empty sequence
Error in Tzone or Pzone: attempt to get argmax of an empty sequence
Records processed!
Evaluating model with MIT-BIH
Accuracy own: 0.9445
Accuracy: 0.9445
F1 Score: 0.7257
              precision    recall  f1-score   support

           0       0.98      0.96      0.97     17780
           1       0.37      0.39      0.38       600
           2       0.96      1.00      0.98      1630
           3       0.40      1.00      0.57       230

    accuracy                           0.94     20240
   macro avg       0.68      0.84      0.73     20240
weighted avg       0.96      0.94      0

## CNN

https://github.com/CVxTz/ECG_Heartbeat_Classification/tree/master

Comandos utilizados para lanzar experimentos (desde la terminal): 

Modelo entrenado con MIT

```bash
python models/train.py --experiment_name MIT-cnn --dataset MIT-BIH --model CNN --path models/saved/MIT-BIH/CNN 
```

**Fíjate que aquí no se utiliza --return_sequences, porque los latidos se pasan al modelo de forma individual, mientras que en el anterior modelo se pasaban en secuencias de 10 latidos**

Modelo entrenado con INCART

```bash
python models/train.py --experiment_name IncartDB-cnn --dataset INCART --model CNN --path models/saved/INCART/CNN
```

In [ ]:
# TODO

model_name = "CNN"
model_module = select_model(model_name, n_classes=4)

for dataset_name in ["MIT-BIH", "INCART"]:
	print("LOADING MODEL TRAINED WITH " + dataset_name + "...")
	path = "/home/nahuel/ecg/generalization/models/saved/"+dataset_name+"/"+model_name+"/best_model.ckpt"
	model = ECGModel.load_from_checkpoint(path, model=model_module)

	for dataset_name in ["MIT-BIH", "INCART"]:
		
		train_dataloader, val_dataloader, test_dataloader = load_data(dataset_name, return_sequences=True)
		print("Evaluating model with " + dataset_name)
		model.evaluate(test_dataloader, "cpu")